In [1]:
import numpy as np
import pandas as pd
from scipy import special

In [2]:
class NN:
    def __init__(self, data):
        self.output_nodes = 10
        self.input_nodes = 784
        self.hidden_nodes = 200
        self.data = data
        
        self.wih = np.random.normal(0.0, pow(self.input_nodes, -0.5), (self.hidden_nodes, self.input_nodes))
        self.who = np.random.normal(0.0, pow(self.hidden_nodes, -0.5), (self.output_nodes, self.hidden_nodes))
        
    def forward_pass(self, pixels):
        hidden_values = np.dot(self.wih, pixels)
        final_hidden = special.expit(hidden_values)
        output_values = np.dot(self.who, hidden_values)
        final_output = special.expit(output_values)
    
    def train(self):
        for index, row in self.data.iterrows():
            example = row.to_numpy()
            label = example[0]
            pixels = np.array(example[1:], ndmin=2).T
            
            hidden_values = np.dot(self.wih, pixels)
            final_hidden = special.expit(hidden_values)
            output_values = np.dot(self.who, hidden_values)
            final_output = special.expit(output_values)
            
            target = np.array([0 if i != label else 1 for i in range(10)], ndmin=2).T
            diff = final_output - target
            
            change_who = np.dot((diff * final_output * (1 - final_output)), final_hidden.T)
            
            local_grad_2 = diff * final_output * (1 - final_output)
            grad_by_y = np.dot(self.who.T, local_grad_2)
            local_grad_1 = grad_by_y * final_hidden * (1 - final_hidden)
            change_wih = np.dot(local_grad_1, pixels.T)
            
            self.who -= change_who
            self.wih -= change_wih

In [3]:
df_images = pd.read_csv("mnist_train.csv", header=None)

In [4]:
df_images

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
num_images = 100
data = df_images[:num_images] 

In [6]:
nn = NN(data)
epochs = 5
for e in range(epochs):
    nn.train()
    print("WIH")
    print(nn.wih)
    print("WHO")
    print(nn.who)

WIH
[[ 0.02937167  0.03768387  0.0113606  ...  0.02016006 -0.00376656
  -0.04425262]
 [ 0.05175884 -0.00802401 -0.01524682 ...  0.01238999 -0.00037605
   0.00830879]
 [ 0.05491484 -0.02652119 -0.03276169 ... -0.00026599  0.02637872
  -0.01046405]
 ...
 [ 0.00840825 -0.03407264  0.01698868 ...  0.0086921  -0.0219848
   0.03008954]
 [ 0.02644724 -0.04484846  0.04847818 ... -0.07026983  0.06365362
  -0.09996821]
 [ 0.01481117 -0.04664464 -0.00989291 ... -0.02932956 -0.06884058
  -0.00674406]]
WHO
[[ 0.16392467 -0.09352552 -0.04132357 ...  0.01451621 -0.10518772
   0.03250131]
 [ 0.0338197   0.02010661  0.03629153 ...  0.06896962  0.02318624
   0.08566711]
 [-0.02427325 -0.04623363 -0.09748755 ...  0.20287608 -0.0715684
  -0.03672443]
 ...
 [-0.03152689  0.04436392  0.06431138 ... -0.09671701  0.05360629
  -0.00494379]
 [-0.04741773 -0.03438702  0.05226348 ... -0.09229911  0.08670142
  -0.0304677 ]
 [ 0.07246454 -0.02911407 -0.0259181  ... -0.070898   -0.08795637
  -0.04138397]]
WIH
[[ 0.0

In [7]:
df_test = pd.read_csv("mnist_test.csv", header=None)

In [8]:
tests = 10
test_data = df_test[:tests]

In [9]:
scorecard = 0
for index, row in test_data.iterrows():
    example = row.to_numpy()
    label = example[0]
    pixels = np.array(example[1:], ndmin=2).T
    
    pred = np.argmax(nn.forward_pass(pixels))
    if pred == label:
        scorecard += 1
print("Test accuracy: " + str(scorecard/tests * 100) + "%")

Test accuracy: 10.0%
